### Python based viewer tool for "Probabilistic Reasoning over time", EDAP01 Artificial Intelligence
This notebook has been provided to you by Alexander Dürr, teaching assistant on the course, spring term 2021. It is based on the ideas and structure of the original Java skeleton for this assignment, provided by Elin A. Topp. Contact us (elin_anna.topp at cs.lth.se) in case you need help!

### Note: Installing and activating widgets for Jupyter Notebook
To be able to display the visualization (dashboard,animations,etc.) you have to initially install the package  if you don't have it yet

### Note: Use Jupyter Lab for programming, Jupyter Notebook for visualization (optional)
This command only enables the extension for jupyter notebook and not in jupyter lab! You can edit from the comfort of jupyter lab though and when you feel like using the widgets just go to

Menu bar > Help > Launch Classic Notebook   

## Here we go... inspecting the models, option 1

### Your main job will be in the file Filters.py and in the last cell of this notebook, this is only to understand the models

In _models_, all the actually interesting stuff is located, and in _Filters_ you should write your own code. Note that the visualization (next cell on inspecting the models, option 2) assumes to have access to an object of type _Localizer_ which in turn expects the filtering / smoothing to happen in _Filters.HMM_Filters.filter(sensorR)_. This means that IF you want to make use of the visualisation in grid-view (below!) also for the actual tracking, you MUST implement the filtering in _Filters.HMM_Filter.filter(sensorR)_ (or make changes in _Localizer.Localizer.update()_)



In [ ]:
from models import *
from view_control.Localizer import Localizer
from models.StateModel import StateModel

# Testing the models, e.g., for an 4x8 grid

states = StateModel( 4, 8)
loc = Localizer( states, 1)
tMat = loc.get_transition_model()
sVecs = loc.get_observation_model()
tMat.plot_T()
sVecs.plot_o_diags()
print(sVecs.get_o_reading(0))
print(sVecs.get_o_reading(None))

print(loc.update())


## Here we go again... inspecting the models, option 2

### Your implementation job will still be in the file Filters.py, this is only to understand the models AND to get a glimpse of a tracking run (but it is slow)

In _models_, all the actually interesting stuff is located, and in _Filters_ you should write your own code. Note that this visualization assumes to have access to an object of type Localizer which in turn expects the filtering / smoothing to happen in _Filters.HMM_Filters.filter()_. This means that IF you want to make use of the visualisation in grid-view also for the actual tracking, you MUST implement the filtering in Filters.HMM_Filter.filter() (or make respective changes in _Localizer.Localizer.update()_)

### Your Task 1)
#### Inspect the visualisation of the models by running the cell and experimenting with the GUI, in particular compare and explain the different versions of the sensor model (see more detailed instructions for the task and report)

In [ ]:
# In view_control.Dashboard, there is simply the handling of all the thread based visualisation provided, 
# no changes needed, but feel free...
from view_control.Dashboard import Dashboard



ROWS = 4
COLS = 4

# The dashboard creates a state model of the dimensions given by ROWS and COLS, sets up the respective 
# Transition and Observation models, as well as an instance of class Localizer. The Localizer calls at the 
# moment a stubb method toDo.Filters.HMMFilter.filter(sensorReading), which just sends back the original 
# probability distribution - no filtering is done. It is your task to implement something useful there.

# Non-uniform failure is the default sensor, sensor 0. Change to 1 if that is your default 
# (uniform sensor failure)
sensorType = 0
dash = Dashboard(ROWS, COLS, sensorType)
display(dash.db)


## Write your own "main" here (without the viewer)

### Your task 2)
#### Implement both Forward Filtering and Forward-Backward Smoothing with k = t-5 (a sequence length of 5)
#### Do evaluations (500 steps should be enough also for the bigger grids) according to the following, adapt the number of steps or the size of the grid if needed, but comment on it
#### Provide plots of the Manhattan distance over time

1)
* Forward Filtering with non-uniform sensor failure on 8x8 grid against
* Sensor output only (non-uniform sensor failure, count sensor failures to get the average frequency, but do not count those steps into the avg Manhattan distance) on 8x8 grid 


2)
* Forward Filtering with non-uniform sensor failure on 4x4 grid against
* Forward Filtering with uniform sensor failure on 4x4 grid


3)
* Forward Filtering with non-uniform sensor failure on 16x20 grid against
* Forward Filtering with uniform sensor failure on 16x20 grid


4)
* Forward Filtering with non-uniform sensor failure on 10x10 grid against
* Smoothing (forward-backward smoothing) with k = t-5 (five steps for b) and non-uniform sensor failure on 10x10 grid

#### OBS: obviously, each pair-wise evaluation should be run based on the same true trajectory (cases 1, 2, 3) or same trajectory AND same sensor reading sequence (for case 4).

In [ ]:
# Trash code, just for testing purposes



from attr import s
import numpy as np
from Filters import HMMFilter
from models.StateModel import StateModel
from models.TransitionModel import TransitionModel
from models.ObservationModel_NUF import ObservationModelNUF
from models.ObservationModel_UF import ObservationModelUF
from models.RobotSim import RobotSim
from view_control.Dashboard import Dashboard
from view_control.Localizer import Localizer
import matplotlib.pyplot as plt


def ManhattanDistance(x1, y1, x2, y2):
    """
    Returns the Manhattan Distance between two points
    :param x1: x-coordinate of point 1
    :param y1: y-coordinate of point 1
    :param x2: x-coordinate of point 2
    :param y2: y-coordinate of point 2
    :return: The Distance
    """
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    distance = dx + dy
    return distance


def displayGraph(
    data1,
    nbrCorrect,
    data2,
    nbrCorrect2,
    data1UF,
    nbrCorrect1UF,
    data2UF,
    nbrCorrect2UF,
):
    average1 = sum(data1) / len(data1)
    average2 = sum(data2) / len(data2)
    average1UF = sum(data1UF) / len(data1UF)
    average2UF = sum(data2UF) / len(data2UF)

    fig, axs = plt.subplots(2, 2, figsize=(12, 5))

    fig.suptitle("Manhattan Distance for NUF above and UF below")
    axs[0][0].plot(data1)
    axs[0][0].set_title("Filtered Data")
    axs[0][0].set_xlabel(
        " average: "
        + str(average1)
        + " Correct Guesses: "
        + str(round(nbrCorrect / len(data1) * 100, 2))
        + "%"
    )
    axs[0][0].set_ylabel("Distance")

    axs[1][0].plot(data2)
    axs[1][0].set_title("Smoothed Data")
    axs[1][0].set_xlabel(
        " average: "
        + str(average2)
        + " Correct Guesses: "
        + str(round(nbrCorrect2 / len(data2) * 100, 2))
        + "%"
    )
    axs[1][0].set_ylabel("Distance")

    axs[0][1].plot(data1UF)
    axs[0][1].set_title("Filtered Data UF")
    axs[0][1].set_xlabel(
        " average: "
        + str(average1UF)
        + " Correct Guesses: "
        + str(round(nbrCorrect1UF / len(data1UF) * 100, 2))
        + "%"
    )
    axs[0][1].set_ylabel("Distance")

    axs[1][1].plot(data2UF)
    axs[1][1].set_title("Smoothed Data UF")
    axs[1][1].set_xlabel(
        " average: "
        + str(average2UF)
        + " Correct Guesses: "
        + str(round(nbrCorrect2UF / len(data2UF) * 100, 2))
        + "%"
    )
    axs[1][1].set_ylabel("Distance")
    plt.show()


def main():
    REPS = 500
    ROWS = 10
    COLS = 10
    nbr_correct1 = 0
    nbr_correct2 = 0
    nbr_correct1UF = 0
    nbr_correct2UF = 0
    avg_error1 = 0
    avg_error2 = 0
    avg_error1UF = 0
    avg_error2UF = 0
    total_error1 = 0
    total_error2 = 0
    total_error1UF = 0
    total_error2UF = 0
    error_array1 = []
    error_array2 = []
    error_array1UF = []
    error_array2UF = []

    # Initialize models and filters
    state_model = StateModel(ROWS, COLS)
    transition_model = TransitionModel(state_model)
    NUF = ObservationModelNUF(state_model)
    UF = ObservationModelUF(state_model)

    nbr_of_states = state_model.get_num_of_states()
    probs1 = np.ones(nbr_of_states) / nbr_of_states
    probs2 = np.ones(nbr_of_states) / nbr_of_states
    probs1UF = np.ones(nbr_of_states) / nbr_of_states
    probs2UF = np.ones(nbr_of_states) / nbr_of_states
    true_pos = 0
    sensor_pos1 = None

    HMMF1 = HMMFilter(probs1, transition_model, NUF, state_model)
    HMMF2 = HMMFilter(probs2, transition_model, NUF, state_model)
    HMMFUF1 = HMMFilter(probs1, transition_model, UF, state_model)
    HMMFUF2 = HMMFilter(probs2, transition_model, UF, state_model)

    robot_sim = RobotSim(true_pos, state_model)

    last_five = []
    last_five_UF = []
    for _ in range(5):
        true_pos_smoothing = robot_sim.move_once(transition_model)
        true_pos_smoothing_UF = true_pos_smoothing
        sensor_pos1 = robot_sim.sense_in_current_state(NUF)
        sensor_pos2 = robot_sim.sense_in_current_state(UF)

        last_five.append([true_pos_smoothing, sensor_pos1])
        last_five_UF.append([true_pos_smoothing_UF, sensor_pos2])

    for m in range(REPS):
        true_pos = robot_sim.move_once(transition_model)
        sensor_pos1 = robot_sim.sense_in_current_state(NUF)
        sensor_pos2 = robot_sim.sense_in_current_state(UF)
        probs1 = HMMF1.filter(sensor_pos1)
        probs1UF = HMMFUF1.filter(sensor_pos2)

        last_five.append([true_pos, sensor_pos1])
        last_five_UF.append([true_pos, sensor_pos2])
        true_pos_smoothing, sensor_pos1 = last_five[0]
        true_pos_smoothing_UF, sensor_pos2 = last_five_UF[0]
        last_five = last_five[1:]
        last_five_UF = last_five_UF[1:]
        probs2 = HMMF2.smoothFilter(last_five, sensor_pos1)
        probs2UF = HMMFUF2.smoothFilter(last_five_UF, sensor_pos2)

        fPositions1 = probs1.copy()
        fPositions2 = probs2.copy()
        fPositions1UF = probs1UF.copy()
        fPositions2UF = probs2UF.copy()

        for state in range(0, state_model.get_num_of_states(), 4):
            fPositions1[state : state + 4] = sum(fPositions1[state : state + 4])
            fPositions2[state : state + 4] = sum(fPositions2[state : state + 4])
            fPositions1UF[state : state + 4] = sum(fPositions1UF[state : state + 4])
            fPositions2UF[state : state + 4] = sum(fPositions2UF[state : state + 4])

        estimate1 = state_model.state_to_position(int(np.argmax(fPositions1)))
        estimate2 = state_model.state_to_position(int(np.argmax(fPositions2)))

        estimate1UF = state_model.state_to_position(int(np.argmax(fPositions1UF)))
        estimate2UF = state_model.state_to_position(int(np.argmax(fPositions2UF)))

        tsX1, tsY1 = state_model.state_to_position(true_pos)
        tsX2, tsY2 = state_model.state_to_position(true_pos_smoothing)
        eX1, eY1 = estimate1
        eX2, eY2 = estimate2

        tsX1UF, tsY1UF = state_model.state_to_position(true_pos)
        tsX2UF, tsY2UF = state_model.state_to_position(true_pos_smoothing_UF)
        eX1UF, eY1UF = estimate1UF
        eX2UF, eY2UF = estimate2UF

        if eX1 == tsX1 and eY1 == tsY1:
            nbr_correct1 += 1

        if eX2 == tsX2 and eY2 == tsY2:
            nbr_correct2 += 1

        if eX1UF == tsX1UF and eY1UF == tsY1UF:
            nbr_correct1UF += 1

        if eX2UF == tsX2UF and eY2UF == tsY2UF:
            nbr_correct2UF += 1

        error1 = abs(tsX1 - eX1) + abs(tsY1 - eY1)
        error2 = abs(tsX2 - eX2) + abs(tsY2 - eY2)
        total_error1 += error1
        total_error2 += error2
        error_array1.append(error1)
        error_array2.append(error2)

        error1UF = abs(tsX1UF - eX1UF) + abs(tsY1UF - eY1UF)
        error2UF = abs(tsX2UF - eX2UF) + abs(tsY2UF - eY2UF)
        total_error1UF += error1UF
        total_error2UF += error2UF
        error_array1UF.append(error1UF)
        error_array2UF.append(error2UF)

    avg_error1 = total_error1 / REPS
    avg_error2 = total_error2 / REPS

    avg_error1UF = total_error1UF / REPS
    avg_error2UF = total_error2UF / REPS

    print("Average Error for Filter 1 UF:", avg_error1UF)
    print("Average Error for Filter 2 UF:", avg_error2UF)
    print("Number of Correct Estimations for Filter 1 UF:", nbr_correct1UF)
    print("Number of Correct Estimations for Filter 2 UF:", nbr_correct2UF)

    print("Average Error for Filter 1:", avg_error1)
    print("Average Error for Filter 2:", avg_error2)
    print("Number of Correct Estimations for Filter 1:", nbr_correct1)
    print("Number of Correct Estimations for Filter 2:", nbr_correct2)

    displayGraph(
        error_array1,
        nbr_correct1,
        error_array2,
        nbr_correct2,
        error_array1UF,
        nbr_correct1UF,
        error_array2UF,
        nbr_correct2UF,
    )


if __name__ == "__main__":
    main()
